# Data Download

In [1]:
from Korpora import Korpora
import pickle 

In [2]:
# Korpora.fetch('namuwikitext')

In [3]:
# corpus = Korpora.load('namuwikitext')
# with open('kor.p', 'wb') as f:
#     pickle.dump(corpus, f)

In [4]:
with open('kor.p', 'rb') as f:
    corpus = pickle.load(f)

# data preprocess

In [7]:
import re
import mecab
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim

import torchtext
import sys
sys.path.append('../source')
from txt_cleaner.clean.master import MasterCleaner
from txt_cleaner.utils import *
from torch8text.data import Vocab, Field

# sentence tokenize

In [20]:
from nltk.tokenize import sent_tokenize
sent_tokenize('안녕하세요? 반갑습니다. 오늘은 엘모를 구현보겠습니다?')

['안녕하세요?', '반갑습니다.', '오늘은 엘모를 구현보겠습니다?']

## field 1: mecab 사용 field

In [21]:
pos = mecab.MeCab()

def tokenize_pos(inp):
    if type(inp) == str:
        return pos.morphs(inp)
    if type(inp) == list:
        return [tokenize_pos(i) for i in inp]

In [22]:
tokenize_pos('안녕하세요'), tokenize_pos(['안녕하세요', '안녕?']), tokenize_pos([['안녕하세요'], ['안녕?']])

(['안녕', '하', '세요'],
 [['안녕', '하', '세요'], ['안녕', '?']],
 [[['안녕', '하', '세요']], [['안녕', '?']]])

### 수인님 cleaner 

In [23]:
config = json_reader('../source/txt_cleaner/cleaner_config.json')

size 1 dictionary is read from ../source/txt_cleaner/cleaner_config.json


In [24]:
config['minimum_space_count'] = 2
config

{'minimum_space_count': 2}

In [25]:
cleaner = MasterCleaner(config)
cleaner.cleaning('안녕하세요? 반갑습니다! 행복하세요~**')

'안녕하세요? 반갑습니다! 행복하세요'

In [26]:
tokenize_pos(['안녕하세요', '안녕'])

[['안녕', '하', '세요'], ['안녕']]

In [27]:
def flatten(S):
    if S == []:
        return S
    if isinstance(S[0], list):
        return flatten(S[0]) + flatten(S[1:])
    return S[:1] + flatten(S[1:])

In [30]:
mecab_field = Field(tokenize = tokenize_pos, 
                    preprocessing = lambda e: cleaner.cleaning(e),
                    init_token = False,
                    eos_token = False
                )

In [31]:
flatten([[['dks','df']]])

['dks', 'df']

In [34]:
train = [sent_tokenize(text) for text in corpus.train.texts if cleaner.cleaning(text)]
train

[['REOL의 앨범 Σ의 일곱번째 트랙으로, 작곡의 원안을 레오루가 담당했다.',
  '기가P는 후렴구의 멜로디를 위주로 작업했다고 하는데 가사와 잘 어울리는 우울하면서도 침착한 멜로디가 어우러져 깊은 여운을 남긴다.',
  '이번 앨범 내에서도 1, 2위를 다툴 정도로 어두운 가사로, 음악의 주제는 사별 ( 死別 ) .',
  '신님이 된 날이라는 제목과 연관지어 생각해보면 꽤나 의미심장하다.',
  '전 소절을 남겨진 사람과 떠난 사람 양쪽의 시점으로 해석 가능하다는 것도 특필할만한 점.'],
 ['羽衣小町\n《아이돌 마스터 신데렐라 걸즈》에 등장하는 코바야카와 사에, 시오미 슈코로 이루어진 2인 유닛.',
  "통칭 '슈사에'.",
  '둘 다 교토가 고향이고 일본 전통 컨셉 아이돌이라는 공통점이 있다.',
  '또한 이 둘은 아이돌을 하는데 있어서 집안과의 갈등을 포함하고 있다는 공통점도 있다.',
  "사에는 엄격한 부모님의 반대를 무릅쓰고 상경, 슈코는 화과자 집 가업을 잇기 싫어서 가출 ( ... )\n모바마스에서 둘의 상호 아이돌 토크가 있고 '하고로모코마치'라는 유닛명을 달고 나와 여러 이벤트에서 출연하고 있다.",
  "유닛명은 '날개옷 미녀'라는 뜻이다.",
  'MAGIC HOUR SP 9화에서 슈코와 사에 단 둘이서만 나와서 직접 유닛명을 언급했다.# 신데메이션 25화에서 사에, 슈코, 아즈키, 카코가 함께 행사를 꾸리는 장면이 스치듯 지나갔다.',
  '신데렐라 걸즈 극장 애니 5화에서 사에, 슈코가 카오루를 데리고 꽃구경을 갔다.',
  '2016년 10월 15일에 열린 신데마스 4thLIVE SSA 공연에서 사에와 슈코의 성우가 青の一番星를 듀엣으로 부르는 무대가 있었다.']]

In [32]:
mecab_field.build_vocab(train)
mecab_field.preprocess('안녕하세요? 룰루랄라 ㅇㅇㄹ')

['안녕', '하', '세요', '?', '룰루랄라']

In [19]:
train

['REOL의 앨범 Σ의 일곱번째 트랙으로, 작곡의 원안을 레오루가 담당했다. 기가P는 후렴구의 멜로디를 위주로 작업했다고 하는데 가사와 잘 어울리는 우울하면서도 침착한 멜로디가 어우러져 깊은 여운을 남긴다.\n이번 앨범 내에서도 1, 2위를 다툴 정도로 어두운 가사로, 음악의 주제는 사별 ( 死別 ) . 신님이 된 날이라는 제목과 연관지어 생각해보면 꽤나 의미심장하다. 전 소절을 남겨진 사람과 떠난 사람 양쪽의 시점으로 해석 가능하다는 것도 특필할만한 점.',
 "羽衣小町\n《아이돌 마스터 신데렐라 걸즈》에 등장하는 코바야카와 사에, 시오미 슈코로 이루어진 2인 유닛. 통칭 '슈사에'. 둘 다 교토가 고향이고 일본 전통 컨셉 아이돌이라는 공통점이 있다. 또한 이 둘은 아이돌을 하는데 있어서 집안과의 갈등을 포함하고 있다는 공통점도 있다. 사에는 엄격한 부모님의 반대를 무릅쓰고 상경, 슈코는 화과자 집 가업을 잇기 싫어서 가출 ( ... )\n모바마스에서 둘의 상호 아이돌 토크가 있고 '하고로모코마치'라는 유닛명을 달고 나와 여러 이벤트에서 출연하고 있다. 유닛명은 '날개옷 미녀'라는 뜻이다.\nMAGIC HOUR SP 9화에서 슈코와 사에 단 둘이서만 나와서 직접 유닛명을 언급했다.# 신데메이션 25화에서 사에, 슈코, 아즈키, 카코가 함께 행사를 꾸리는 장면이 스치듯 지나갔다. 신데렐라 걸즈 극장 애니 5화에서 사에, 슈코가 카오루를 데리고 꽃구경을 갔다.\n2016년 10월 15일에 열린 신데마스 4thLIVE SSA 공연에서 사에와 슈코의 성우가 青の一番星를 듀엣으로 부르는 무대가 있었다."]

## field 2:  chr-level field

In [56]:
cleaner.cleaning('아')

''

In [57]:
cleaner = MasterCleaner({'minimum_space_count':0})
chr_field = Field(tokenize = list, 
                 preprocessing = lambda e: cleaner.cleaning(e) if len(e) > 1 else e,
                  init_token = False,
                  eos_token = False,
                )

In [58]:
chr_field.build_vocab(train)

In [59]:
token_chr = chr_field.preprocess('안녕하세요')
chr_field.process(token_chr)

[72, 1, 18, 1, 1]

# dataset, dataloader
## 헷갈리는 부분
bi-directional LSTM을 쓸건데 이게 다음 단어 예측하는 LM만 데이터를 구성하면 되나? 아니면 뒤에서부터 앞의 단어를 예측하는 LM도 구성해서 concat해야 하나? -> 일단 전자라고 생각하고 함

In [60]:
from collections import namedtuple  
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

class ELMoDataset(Dataset):
    def __init__(self, src, mecab_field, chr_field):
        self.src = src
        self.mecab_field = mecab_field
        self.chr_field = chr_field
        self.named_tuple = namedtuple('data', ['src', 'trg', 'src_chr'])
        
    def __len__(self):
        return len(self.src)
    
    def __getitem__(self, idx):
        return self.named_tuple(self.getitem(idx), self.getitem(idx)[1:], self.getitem(idx, is_char=True))
    
    def getitem(self, idx, is_char=False):
        data = self.src[idx]
        tokenize_data = self.mecab_field.preprocess(data)
        if is_char:
            chrs = chr_field.preprocess(tokenize_data)
            pad_chrs = self.chr_field.pad_process(tokenize_data)
            return pad_chrs
        return torch.Tensor(self.mecab_field.vocab.stoi(tokenize_data)).long()

In [61]:
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence

In [62]:
ds = ELMoDataset(train, mecab_field, chr_field)

In [63]:
max_len = max([len(_) for _ in mecab_field.vocab.stoi_dict])

In [64]:
max_len

6

In [65]:
X  = [torch.tensor([[72,  0,  0,  0,  0]]), torch.tensor([[0, 0, 0, 0, 0]])]

In [66]:
torch.cat(X)

tensor([[72,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0]])

In [67]:
chr_field.max_len

5

In [68]:
data = '안녕하세요 반갑습니ek edd'
token_data = mecab_field.preprocess(data)
print(token_data)
token_chr_data = chr_field.preprocess(token_data)
print(token_chr_data)
process_chr = chr_field.pad_process(token_chr_data)
print(process_chr)

[['안녕', '하', '세요', '반갑', '습니', 'ek', 'edd']]
[[['안', '녕'], ['하'], ['세', '요'], ['반', '갑'], ['습', '니'], [], []]]


TypeError: unhashable type: 'list'

In [ ]:
process_chr.shape

In [ ]:
ds[1]

In [ ]:
for _ in ds:
    print(_.src_chr)
    print(_.src)
    print(_.trg)
    break

In [ ]:
def pad_collate(batch):
    (src, trg, src_chr) = zip(*batch)
    named_tuple = namedtuple('data', ['src', 'trg', 'src_chr'])
    src_pad = pad_sequence(src, batch_first=True, padding_value=0)
    trg_pad = pad_sequence(trg, batch_first=True, padding_value=0)
    src_chr_pad = pad_sequence(src_chr, batch_first=True, padding_value=0)
    return named_tuple(src_pad, trg_pad, src_chr_pad)

In [ ]:
# def pack_pad_collate(batch):
#     (src, trg) = zip(*batch)
#     src_len = torch.Tensor([len(s) for s in src])
#     trg_len = torch.Tensor([len(t) for t in trg])
#     named_tuple = namedtuple('data', ['src', 'trg'])
#     src_pad = pad_sequence(src, batch_first=True, padding_value=0)
#     trg_pad = pad_sequence(trg, batch_first=True, padding_value=0)
#     src_pack = pack_padded_sequence(src_pad, lengths=src_len, batch_first=True, enforce_sorted=False)
#     trg_pack = pack_padded_sequence(trg_pad, lengths=trg_len, batch_first=True, enforce_sorted=False)
#     return named_tuple(src_pack, trg_pack)

In [ ]:
dl = DataLoader(ds, batch_size = 16, collate_fn = pad_collate)
for _ in dl:
    print(_.src.data.shape)
    print(_.trg.data.shape)
    print(_.src_chr.data.shape)
    break

In [ ]:
class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, 
                 dropout, pad_idx):
        
        super().__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        self.conv_0 = nn.Conv2d(in_channels = 1, 
                                out_channels = n_filters, 
                                kernel_size = (filter_sizes[0], embedding_dim)) 
        
        self.conv_1 = nn.Conv2d(in_channels = 1, 
                                out_channels = n_filters, 
                                kernel_size = (filter_sizes[1], embedding_dim))

        
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
                
        #text = [batch size, sent len]
        
        embedded = self.embedding(text)
        # torch.Size([2, 243, 5, 1024])
        #embedded = [batch size, 1, sent len, emb dim]
        
        conved_0 = F.relu(self.conv_0(embedded).squeeze(3))
        conved_1 = F.relu(self.conv_1(embedded).squeeze(3))

        #conved_n = [batch size, n_filters, sent len - filter_sizes[n] + 1]
        
        pooled_0 = F.max_pool1d(conved_0, conved_0.shape[2]).squeeze(2)
        pooled_1 = F.max_pool1d(conved_1, conved_1.shape[2]).squeeze(2)
        
        #pooled_n = [batch size, n_filters]
#         print(pooled_0.shape)
        cat = self.dropout(torch.cat((pooled_0, pooled_1), dim = -1))

        #cat = [batch size, n_filters * len(filter_sizes)]
            
        return self.fc(cat)

In [ ]:
VOCAB_SIZE = len(chr_field.vocab)
EMBEDDING_DIM = 1024
N_FILTERS = 1
FILTER_SIZES = (1, 2)
PAD_IDX = chr_field.vocab.stoi_dict['<PAD>']
SPECIAL_TOKENS = chr_field.vocab.special_tokens
SPECIAL_TOKENS_INDEX = chr_field.vocab.special_tokens_idx
CHR_DICT = chr_dict
OUTPUT_DIM = 1024
DROPOUT = 0.5

In [ ]:
cnn = CNN(VOCAB_SIZE, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT, PAD_IDX)

In [ ]:
class LSTM_LM(nn.Module):
    def __init__(self, input_dim, output_dim, hid_dim, n_layers, dropout, bidirectional):
        super().__init__()
        
        self.output_dim = output_dim
        self.input_dim = input_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        self.num_dircetions = 2 if bidirectional else 1
        
        self.lstm = nn.LSTM(input_dim, hid_dim, n_layers, dropout = dropout, bidirectional = bidirectional)
        
        self.fc_out = nn.Linear(hid_dim * self.num_dircetions, output_dim)
        
        self.dropout = nn.Dropout(dropout)
    
    def init_hidden(self):
        # (num_layers * num_directions, batch, hidden_size)
        return torch.zeros(self.n_layers * self.num_dircetions, 1, self.hid_dim)
    
    def forward(self, input):
#         print(f'input shape : {input.shape}') # seqlen, batch, hid_dim(output_dim of cnn)
        output, (hidden, cell) = self.lstm(input)
#         print(f'output shape : {output.shape}') # ouput shape :(seq_len, batch, num_directions * hidden_size)  
        
        prediction = self.fc_out(output)
#         print(f'prediction shape {prediction.shape}') # seq len, batchsize, trg_dim
        return prediction, hidden, cell

In [ ]:
OUTPUT_DIM

In [ ]:
INPUT_DIM = OUTPUT_DIM
PREDICT_DIM = len(mecab_field.vocab)
HID_DIM = 1024
N_LAYERS = 2
DROPOUT = 0.5
BIDIRECTIONAL = True
TRG_PAD_IDX = mecab_field.vocab.stoi_dict['<PAD>']

In [ ]:
rnn = LSTM_LM(INPUT_DIM, PREDICT_DIM, HID_DIM, N_LAYERS, DROPOUT, BIDIRECTIONAL)

`<sos>`토큰이랑 `<eos>` 토큰은 어떻게 CNN처리 해야하지?  -> 일단 빼는걸로 처리

## 한글이랑 영어랑 다른점 : 영어는 3char이하인 단어가 별로 없는데 한글은 1~2개로 많이 끊겨서 conv연산 하기가 애매함 

In [ ]:
criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)
import torch.optim as optim

optimizer = optim.Adam(cnn.parameters(), lr=0.0005)

In [ ]:
def initialize_weights(m):
    if hasattr(m, 'weight') and m.weight.dim() > 1:
        nn.init.xavier_uniform_(m.weight.data)

In [ ]:
cnn.apply(initialize_weights);
rnn.apply(initialize_weights);

In [ ]:
epoch_loss = []
n_pass = 0
for epoch in range(10):
    optimizer.zero_grad()
    for src, trg, src_chr in dl:
        src_chr = src_chr.permute(1, 0, 2) # 토큰 별 캐릭터가 먼저 나오도록

        for idx, src_c in enumerate(src_chr):
            output, hidden, cell = rnn(features.unsqueeze(1))
            try:
                loss = criterion(output.squeeze(1), trg[:, idx])
                loss.backward()
                optimizer.step()
                
            except:
                pass
    epoch_loss += [loss.item()]

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(epoch_loss)

# 해야 할 일 : 1) init hidden 2) CNN + RNN 감싸기
1) 배치별로 토큰 내에 있는 캐릭터 글자에 따라 CNN길이가 다른데 어떻게 처리하지? -> 패딩으로 처리함..근데 이게 맞는지 모르겠다